# Práctica con LLM - Prompt Engineer 101

Blog Aprende Machine Learning en español - www.aprendemachinelearning.com

In [3]:
# Recuerda instalar el paquete de OpenAI con `pip install openai` aunque utilicemos un modelo local como LLama 2 o Falcon, etc.
# En esta notebook uso la versión de libreria 1.13.3
# Si tienes la API de OpenAI de pago también puedes usar este código, deberás especificar tu propio API KEY y el modelo que deseas usar.

from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
modelo = "local-model" # si tienes la API de pago: gpt-4 , gpt-3.5, etc


## Funcion para obtener las respuestas

In [67]:

def get_completion(prompt:str, model:str=modelo, temperature:float=0):
    messages = [{
            "role": "system",
            "content": "Eres un asistente en español y ayudas respondiendo con la mayor exactitud posible.",
        },
        {"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

## Pedir Tareas de forma sencilla
### Uso de Delimitadores

In [73]:
text = f"""
La sonda Voyager 1 ha vuelto a dar señales de vida. Hace poco te contaba en un vídeo largo de mi canal que los ingenieros de la NASA \
habían perdido completamente la comunicación con la sonda Voyager 1, que lleva haciendo ciencia durante décadas. \
Tenemos buenas noticias y es que nos están llegando señales coherentes de esta sonda. El día 3 de este mes nos llegó una comunicación \
desde el espacio, que aun no siendo legible, tenía buena pinta, así que alguien de la NASA la decodificó y resulta que su contenido es importante. \
Se trata de un volcado completo del sistema de datos de vuelo, que recoge los datos de todos los Instrumentos y sensores que aún están \
funcionando, de las variables internas de la sonda y otros datos adicionales. \
Por supuesto esto ha dado esperanzas al equipo de la Voyager 1, que ahora se está planteando intentar recuperar la comunicación con \
la sonda de alguna forma.
"""

prompt = f"""
Debes resumir en muy pocas palabras el siguiente texto delimitado por triple comilla simple: ```{text}```
"""
response = get_completion(prompt)
print(response)

 La sonda espacial Voyager 1 ha vuelto a enviar señales de vida después de perderse las comunicaciones. Los ingenieros de la NASA recibieron un volcado completo del sistema de datos de vuelo, que incluye información de todos los instrumentos y sensores funcionando, así como otros datos internos. Esta información ha dado a los científicos esperanza de recuperar el contacto con la sonda.


### Formato de Salida

In [ ]:
prompt = f"""
Genera una lista con tres títulos inventados de libros sobre lo bueno que es volar y el nombre del autor.
Provee una salida en formato JSON con las siguientes claves: libro_id, titulo, autor, año.
"""
response = get_completion(prompt)
print(response)

 {
  "libro_1": {
    "titulo": "Volar con alas de acero",
    "autor": "Juan Pérez",
    "año": 2023
  },
  "libro_2": {
    "titulo": "El secreto del vuelo",
    "autor": "María Gómez",
    "año": 2021
  },
  "libro_3": {
    "titulo": "Viajes en el cielo",
    "autor": "Carl<|im_start|> Mendoza",
    "año": 2022
  }
}


### Pedir que Siga instrucciones

In [ ]:
text_1 = f"""
Instrucciones para dar cuerda al reloj

Allá al fondo está la muerte, pero no tenga miedo. Sujete el reloj con una mano, tome con dos dedos la llave de la cuerda, remóntela suavemente. 
Ahora se abre otro plazo, los árboles despliegan sus hojas, las barcas corren regatas, el tiempo como un abanico se va llenando de sí mismo y de él brotan el aire, las brisas de la tierra, la sombra de una mujer, el perfume del pan.
¿Qué más quiere, qué más quiere? Atelo pronto a su muñeca, déjelo latir en libertad, imítelo anhelante. El miedo herrumbra las áncoras, cada cosa que pudo alcanzarse y fue olvidada va corroyendo las venas del reloj, gangrenando la fría sangre de sus rubíes.
Y allá en el fondo está la muerte si no corremos y llegamos antes y comprendemos que ya no importa.
"""

prompt = f"""
Te pasaré un texto delimitado por triple comillas. 
Si contiene una secuencia de instrucciones, re-escribe esas instrucciones siguiendo el siguiente formato:

Paso 1 - ...
Paso 2 - …
…
Paso N - …

Si el texto no contiene instrucciones, simplemente responde \"No hay instrucciones.\"

\"\"\"{text_1}\"\"\"
"""

response = get_completion(prompt)
print("Respuesta:")
print(response)

Respuesta:
 "Paso 1 - Sujete el reloj con una mano."
"Paso 2 - Tome con dos dedos la llave de la cuerda, remóntela suavemente."
"Paso 3 - Ahora se abre otro plazo, los árboles despliegan sus hojas, las barcas corren regatas, el tiempo como un abanico se va llenando de sí mismo y de él brotan el aire, las brisas de la tierra, la sombra de una mujer, el perfume del pan."
"Paso 4 - ¿Qué más quiere, qué más quiere? Atelo pronto a su muñeca, déjelo latir en libertad, imítelo anhelante."
"Paso 5 - El miedo herrumbra las áncoras, cada cosa que pudo alcanzarse y fue olvidada va corroyendo las venas del reloj, gangrenando la fría sangre de sus rubíes."
"Paso 6 - Y allá en el fondo está la muerte si no corremos y llegamos antes y comprendemos que ya no importa."


### Si no cumple lo que pedimos, dar otra salida

In [ ]:
text_1 = "Los amigos son esos seres que siempre están ahí, aunque lleves años sin verlos. A los que les puedes decir lo que piensas sin temor a perderlos. \
Los amigos que tienes que envolver en papel cebolla para que perduren, esos no son amigos, son el propio papel que rellena tú vida."
prompt = f"""
Te pasaré un texto delimitado por triple comillas. 
Si contiene una secuencia de instrucciones, re-escribe esas instrucciones siguiendo el siguiente formato:

Paso 1 - ...
Paso 2 - …
…
Paso N - …

Pero si el texto no contiene instrucciones, debes responder \"No hay instrucciones.\" y no escribir ninguna lista de pasos

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Respuesta:")
print(response)

Respuesta:
 "No hay instrucciones."


## Ejemplo de Few shot prompting

Le damos al modelo un ejemplo de cómo queremos que continúe escribiendo

In [ ]:
prompt = f"""
Tu tarea es responder siguiendo el mismo estilo que ves a continuación.

<Juan>: Esa nube tiene forma de triángulo.

<Músico>: Me recuerda al disco de Pink Floyd.

<Juan>: Esa otra nube tiene forma de lengua.

<Músico>: Me recuerda al disco de los Rolling Stones.

<Juan>: Esa tiene forma de círculo.

<Músico>:"""
response = get_completion(prompt)
print(response)

 Esa última nube que mencionas, ¿tiene alguna forma en particular?

<Juan>: No, no tiene una forma específica, pero se parece a un disco de música.

<Músico>: Ah, entonces me recuerda al disco de los Beatles.


## Dar Tiempo de Reflexion al modelo
### Antes de contestar

In [ ]:
text = "Juan era un niño aventurero, muy curioso y sobre todo soñador. Una tarde, después de haber estado jugando durante horas en el parque con sus amigos, \
    y después de haber cenado con Mamá María y Papá Jorge, Mamá decidió acompañar a Juan a su habitación, ya que era hora de ir a la cama. \
La habitación de Juan estaba decorada con pósteres de barcos y mapas antiguos, y su cama estaba rodeada de juguetes y libros de aventuras de piratas. \
    Mamá se acercó a Juan, le dio un fuerte abrazo y le deseó buenas noches. Al poco tiempo, Juan se quedó profundamente dormido."

prompt_2 = f"""
Tu tarea es realizar las siguientes acciones: 
1 - Resumir el texto delimitado con <> en una breve oración.
2 - Traducir el texto a Italiano.
3 - Listar los nombres de personas del texto.
4 - Crear un objecto JSON que contenga las claves: resumen_italiano, nombres.

Usa el siguiente formato:
Resumen: <resumen>
Traducción: <resumen traducido>
Nombres: <Lista de nombres encontrados>
Salida JSON: <Json con las claves resumen_italiano y nombres>

Texto: <{text}>
"""

response = get_completion(prompt_2)

print("\nRespuesta:")
print(response)


Completion for prompt 2:
 Resumen: Juan era un niño aventurero, muy curioso y sobre todo soñador. Después de haber estado jugando durante horas en el parque con sus amigos, después de haber cenado con Mamá María y Papá Jorge, Mamá decidió acompañar a Juan a su habitación, ya que era hora de ir a la cama. La habitación de Juan estaba decorada con pósteres de barcos y mapas antiguos, y su cama estaba rodeada de juguetes y libros de aventuras de piratas. Mamá se acercó a Juan, le dio un fuerte abrazo y le deseó buenas noches. Al poco tiempo, Juan se quedó profundamente dormido.
Traducción: Juan era un bambino avventuroso, molto curioso e soprattutto sognoioso. Dopo aver trascorso ore in giuoco nel parco con i suoi amici, dopo avere cenato con Mamma Maria e Papà Jorge, Mamma decise di accompagnare Juan alla sua camera, poiché era ora di andare a letto. La camera di Juan era decorata con poster di navi e mappe antichi, e la culla era circondata da giocattie e libri d'avventura pirata. Mamm

### Trabaja internamente la solución

In [108]:
prompt = f"""
Eres un profesor de matemáticas muy riguroso y tu tarea es determinar si el examen que hizo el estudiante es correcto o incorrecto.
Para resolver el problema deberás:
- Primero, trabaja en tu propia solución al problema y calcula el valor Total final. 
- Presta atención al valor de IVA del Enunciado. Si no se usa bien habrá fallo en los cálculos.
- Compara tu solución con la del estudiante y evalúa si la solución del estudiante es correcta o no. 
No decidas si el estudiante ha acertado o fallado hasta haber realizado las operaciones tu mismo.

Enunciado:
```
Al comprar un Televisor que valía $250 nos hacen una rebaja del 4%. 
Luego de rebajarlo tenemos que añadir el 20% de IVA.
¿Cuánto pagamos por el televisor?
``` 
Solución del estudiante:
```
1. Calculo el 4% de 250: (4 * 250) / 100 = (1000 / 100) = 10
2. Resto la rebaja: (250 - 10) = 240
3. Añado el 10% de IVA al valor rebajado: (10 * 240) / 100 = (2400/100) = 24
4. Sumo el Total: 240 + 24 = 264
Respuesta: pagamos $264 por el televisor.
```

Solución Real:
```
Desarrolla aqui tu solución con detalle. 
Determina si la solución del estudiante tiene fallo.
```
"""
response = get_completion(prompt)
print(response)

 Primero, calculamos el valor de rebaja: (250 * 4%) = 10. Luego, restamos la rebaja al precio original: 250 - 10 = 240. A continuación, añadimos el IVA del 20%: (240 * 20%) / 100 = 48. Finalmente, sumamos el valor de rebaja y el IVA al precio rebajado: 240 + 48 = 288.

La solución del estudiante tiene un error en la operación de cálculo del IVA. El IVA debería ser 240 * 20% / 100, lo cual da como resultado 48. Si se añade este valor al precio rebajado (240), se obtendría un total de $288.

La solución correcta es: "pagamos $288 por el televisor".


## Cuidado con las alucionaciones...

In [ ]:
prompt = "Cuentame como se usa el ultimo producto de Tubble y porque es tan valioso."
response = get_completion(prompt)
print(response)

 El último producto de Tubble, llamado "Tubble-X", es un potente antioxidante que se utiliza en la industria farmacéutica y de cuidados personales. Tiene una gran cantidad de aplicaciones debido a sus propiedades antioxidantes, antiinflamatorias y anticancerígenas.

Tubble-X se obtiene a través de un proceso de extracción y purificación de los compuestos naturales presentes en la planta Tubble, que crece únicamente en ciertas regiones del mundo. La extracción se lleva a cabo mediante un proceso ecológico y sostenible, sin dañar ni afectar al medio ambiente ni a las futuras generaciones de la planta Tubble.

Una vez extraído, el Tubble-X se utiliza en una amplia gama de productos, desde medicamentos hasta cosméticos y alimentos. Algunos de los usos más comunes incluyen:

1. Medicinas: Se utiliza como ingrediente principal en la formulación de medicamentos para tratar diversas afecciones, como el cáncer, las enfermedades inflamatorias y las enfermedades cardíacas.
2. Cuidados personales:

# Resumen de Texto

In [111]:
prod_review = """
Se mantiene a unas temperaturas ridículas a 0.975-1v @ 2800mhz. Los tres ventiladores a 35-37% no se oyen en absoluto y es más que suficiente para refrigerarla.
El consumo a máximo rendimiento no pasa de 210W.
Tan solo aplícale un undervolt con la curvatura y listo.
Sin duda, 4070 super es lo mejor que Nvidia ha fabricado. Nunca había probado este ensamblador MSI ventus. De momento muy contento con la compra, puedo garantizar que es de altísima calidad.
Mi anterior 3070 gybabyte el núcleo a 80°c y las memorias a 100-105. I cluso cambíandole los thermalpads... En fin, estoy muy feliz con esta MSI 4070.
"""

In [112]:
prompt = f"""
Tu tarea es generar un breve resumen de una reseña de un producto de un ecommerce.

Resume la reseña que viene a continuación, delimitada por triple comilla, en máximo de 30 palabras.

Reseña: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

 "Muy potente en temperaturas bajas y consumo bajo a 2800mhz, ventiladores silenciosos y undervolt para mejorar. Nvidia 4070 de MSI es una excelente adquisición."


In [113]:
prompt = f"""
Tu tarea es generar un breve resumen de una reseña de un producto de un ecommerce para dar Feedback al departamento de Ventas.

Resume la reseña que viene a continuación, delimitada por triple comilla, en máximo de 30 palabras y centrate en el consumo eléctrico.

Reseña: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

 La tarjeta 4070 de MSI mantiene la temperatura a niveles razonables y el consumo máximo es de 210W. El usuario está muy contento con la compra, ya que su anterior 3070 tenía problemas de temperatura.


# LLM para tareas de NLP

In [114]:
monitor_review = """
El monitor LG 29WP500-B UltraWide impresiona con su relación de aspecto 21:9 y su panel IPS de alta resolución (2560x1080). 
El uso de FreeSync marca una clara diferencia en términos de compatibilidad con tarjetas gráficas y garantiza una visualización fluida. 
En comparación con mi antiguo monitor Eizo, el monitor LG ofrece una fidelidad de color superior con una cobertura sRGB de más del 99%. 
Con dos puertos HDMI y la posibilidad de inclinarlo, también es avanzado en términos de conectividad y facilidad de uso. 
Este monitor es una opción de actualización que merece la pena para disfrutar de una experiencia visual impresionante.
"""

## Análisis de Sentimiento

In [115]:
prompt = f"""
Cuál es el sentimiento de la reseña hecha al producto
que viene a continuación, delimitada por triple comilla?

Reseña: '''{monitor_review}'''
"""
response = get_completion(prompt)
print(response)

 El sentimiento de la reseña es positivo, ya que el usuario menciona las ventajas del producto y lo considera una opción de actualización que merece la pena para disfrutar de una experiencia visual impresionante.


In [116]:
prompt = f"""
Cuál es el sentimiento de la reseña hecha al producto
que viene a continuación, delimitada por triple comilla?

Da tu respuesta en una sóla palabra: "positivo" o "negativo".

Reseña: '''{monitor_review}'''
"""
response = get_completion(prompt)
print(response)

 "positivo"


## Emociones

In [117]:
prompt = f"""
Identifica una lista de emociones del escritor para la siguiente reseña.
No incluyas más de tres emociones en tu lista.
El formato de la lista será de palabras en minúscula separadas por coma.

Reseña: '''{monitor_review}'''
"""
response = get_completion(prompt)
print(response)

 "impresionante", "fidelidad", "avanzado"


## Detección de Cliente enfadado

In [119]:
prompt = f"""
Contesta si el escritor de la siguiente reseña está expresando ira, furia o enojo.
La reseña está delimitada por triple comillas.
Da tu respuesta en una sóla palabra como "si" o "no".

Reseña: '''{monitor_review}'''
"""
response = get_completion(prompt)
print(response)

 "no"


## Extraer Entidades

In [120]:
prompt = f"""
Identifica los siguientes elementos del texto de la reseña:
- Producto comprado por el autor
- Compañía del producto

La reseña está delimitada por triple comillas.
La respuesta deberá ser en un objeto JSON con "Item" y "Marca" como claves. 
Si no se encuentra la información, usa "desconocido" como valor.
Haz tu respuesta lo más corta posible.
  
Reseña: '''{monitor_review}'''
"""
response = get_completion(prompt)
print(response)

 { "Item": "Monitor LG 29WP500-B", "Marca": "LG" }


## Clasificación de Temática

In [121]:
story = """
Una vez hice un garabato de muchos círculos y a lápiz en la mesa de al lado de mi compañero de pupitre. No sé si recuerdas esas mesas verdes que habitaban nuestras aulas de manera continuada año tras año.
He de decir que este año cumpliré 50 tacos, es decir, te estoy hablando del año 1984 aproximadamente, tenía 10 años.

Ni siquiera sé por qué lo hice. Se me ocurrió y le planté un garabato enorme sin que él se diese cuenta.
Se ve que aquel día mi cabeza andaba sola en formato ameba y no se me ocurrió otra cosa que aportar a la humanidad.
Yo era un crío formal, he de decirlo.

Tuve la mala suerte de que el profe vio el tachón en la mesa al rato y dijo todo serio que si nadie asumía la culpa el finde no nos íbamos de excursión.
Joder que mala suerte, era finde de excursión. Eran las míticas excursiones que te ponías nervioso.
Ya sabes, camping, colegueo, cotilleo del pelo que si a fulanito le gusta menganita pero no te chives … y a los 5 minutos ya lo sabía toda la clase.
Estas son las cosas que todos recordamos.
 
Bien, el asunto es que en ese momento no dije nada. Callado como un muerto.
Me fui a casa a comer y a meditar.
Por la tarde, a la vuelta, tenía que darle solución.
 
Realmente no había opción, lo tenía que confesar. Tenía que enfrentarme a mis miedos y asumir mi culpa.
He de decir, que en ningún momento se me pasó por la cabeza fallar a mis compañeros. La idea de que por mi culpa se quedaran sin excursión no asomó en ningún momento como posible opción.
Tampoco contemplé la posibilidad de que el profe fuese de farol. Probablemente fuera así, pero en aquél entonces tenía las preocupaciones de un niño de 10 años.
"""
# Extracto del texto "Lecciones de negocio de un crío de 10 años" de Unai Martinez

In [123]:
prompt = f"""
Determina cinco tópicos que se comentan en el siguiente texto delimitado por triple comillas.

Cada tema será definido por una o dos palabras.

El formato de tu respuesta debe ser una lista separada por comas.

Texto: '''{story}'''
"""
response = get_completion(prompt)
print(response)

 "Garabato", "Mesas verdes", "Años 80", "Excursiones" y "Confesar".


In [125]:
response = response.replace(" y ", ", ")
response.split(sep=',')

[' "Garabato"',
 ' "Mesas verdes"',
 ' "Años 80"',
 ' "Excursiones"',
 ' "Confesar".']

### Detección de temas

In [142]:
topic_list = [
    "excursiones", "playa", "automovilismo", "cliente", "monetario"
]
print(", ".join(topic_list))

excursiones, playa, automovilismo, cliente, monetario


In [143]:
prompt = f"""
Determina para cada elemento de la Lista si es un tema abordado en el texto que viene debajo delimitado por triple comilla simple.

Da tu respuesta como una lista de 0s y 1s para cada elemento. Si el item se detecta en el Texto, pon 1, si no 0.

Lista: {", ".join(topic_list)}

Texto: '''{story}'''

¿Se tratan los temas de la lista en el Texto?
"""
response = get_completion(prompt)
print(response)

 [1, 0, 0, 0, 0]


In [146]:
if ":" in response:
    topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
else:
    response1 = response.replace("[","").replace("]","")
    topic_dict = {i: int(j) for i, j in zip(topic_list, response1.split(sep=','))}

if topic_dict['excursiones'] == 1:
    print("ALERTA: Detectamos que habla sobre excursiones!")

ALERTA: Detectamos que habla sobre excursiones!


# Traducción de textos

In [147]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a coffee```
"""
response = get_completion(prompt)
print(response)

 "Hola, me gustaría pedir un café"


In [149]:
prompt = f"""
Dime en que idioma esta el texto: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

 Este texto está en francés (idioma de Francia). La pregunta se traduce como "¿Cuánto cuesta la lámpara?" en español.


In [151]:
prompt = f"""
Traduce el siguiente texto a Inglés en modo formal e informal: 
'Te quiero enviar un documento de texto'
"""
response = get_completion(prompt)
print(response)

 Formal: "I would like to send you a text document."
Informal: "I'm going to send you a text doc."


## Correccion de Texto o gramática

In [157]:
text = [
  "La niña esta jugando con la perro.",  # error
  "Juan tiene un ordenador.", # ok
  "Ba a acer un largo día.",  # error
  "Hayamos sido tan felices juntos.",  # error
  "El coche es rojo.", # ok
  "Vamos a porbar si hay error de deletreo."  # error deletreo
]
for t in text:
    prompt = f"""
    Corrige ls siguiente oración delimitada por triple comilla simple y reescribe la versión revisada sin error. Si no encuentras error, escribe: "No hay error".
    Oración: ```{t}```"""
    response = get_completion(prompt)
    print(f"Respuesta para '{t}':")
    print(response)
    print("- " * 10)

Respuesta para 'La niña esta jugando con la perro.':
 "La niña está jugando con el perro."
- - - - - - - - - - 
Respuesta para 'Juan tiene un ordenador.':
 No hay error. La oración ya está correcta en términos de gramática y estructura.
- - - - - - - - - - 
Respuesta para 'Ba a acer un largo día.':
 "Vaya a arreglar un largo día."
- - - - - - - - - - 
Respuesta para 'Hayamos sido tan felices juntos.':
 "Hemos estado tan felices juntos."
- - - - - - - - - - 
Respuesta para 'El coche es rojo.':
 "El auto es rojo."
- - - - - - - - - - 
Respuesta para 'Vamos a porbar si hay error de deletreo.':
 "Vamos a probar si hay error de ortografía."
- - - - - - - - - - 


# Un Chatbot sencillo

In [4]:
def get_completion_from_messages(messages, model=modelo, temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    msg = response.choices[0].message
    return msg.content

In [161]:
messages =  [
    {'role':'system', 'content':'Eres un robot amistoso. Responde brevemente lo que se te pide.'},
    {'role':'user', 'content':'Hola, mi nombre es Juan.'}  
    ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

 ¡Hola, Juan! Me encantaría saber qué tipo de información puedo proporcionarme para ayudarte? ¿Puedes darme más detalles sobre tu consulta?


In [162]:
messages =  [
    {'role':'system', 'content':'Eres un robot amistoso. Responde brevemente lo que se te pide.'},
    {'role':'user', 'content':'Sí, ¿puedes decir cúal es mi nombre?'}  
    ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

 Si, puedo recordar que mencionaste tu nombre en la conversación. Asumimos que quieres hablar acerca de ti mismo o a alguien con un nombre similar. A continuación, me invitarías a usar un tono educador, divertido, o simplemente neutro, según sea necesario:

- Tonedo educador: "Voy a asumir que me estás pidiendo información sobre ti mismo, y considerando la forma en que mencionamos su nombre, quizás te guste escuchar algo al respecto".
- Tonedo divertido: "Muy bien! Así es cómo lo recuerdo. ¡Te cuestaba recordarlo? Te has llamado tú mismo! Genial."
- Tono neutro: "Entonces, ¿deseas conocer información sobre ti o a alguien llamado así?"


In [170]:
messages =  [
    {'role':'system', 'content':'Eres un robot amistoso. Responde brevemente lo que se te pide.'},
    {'role':'user', 'content':'Hola, mi nombre es Juan.'},
    {'role':'assistant', 'content': "¡Hola, Juan! Me encantaría saber qué tipo de información puedo proporcionarme para ayudarte? ¿Puedes darme más detalles sobre tu consulta?"},
    {'role':'user', 'content':'Sí, ¿puedes decir cúal es mi nombre?'}
    ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

 ¿Te refieres a ti mismo, Juan? ¡Así lo sabemos ya!


## Creamos un Bot de atención al cliente para venta de Pizzas

In [11]:
context = [ {'role':'system', 'content':"""
Eres PedidosBot, un servicio automatizado que recoge pedidos de pizza de un restaurante.
Primero saludas al cliente, luego recibes su orden
Y luego preguntas si el pedido es para enviar a domicilio o si lo vienen a buscar.
Esperas a tener la orden completa, luego haces un resumen y calculas el precio total final.
Si es envío a domicilio deberás pedir al cliente su dirección.
Responde siendo breve y amistoso pero formal.
El menú incluye:
Pizza pepperoni  12.95 \
Pizza de Jamón y Queso   10.95 \
Pizza con huevo   11.95 \
Patatas fritas 4.50 \
Ensalada griega 7.25 \
Extras: \
extra de Queso 2.00, \
Setas 1.50 \
Bebidas: \
Zumo de Naranja 3.00 \
sprite 3.00 \
Agua 5.00 \
"""} ]


In [12]:
# Creamos un loop de consultas al bot, para salir escribe "salir" ó un mensaje vacío.
# NOTA: Si estás en VSCODE la caja de texto aparece en la parte de arriba de la pantalla.
texto_usuario = "inicio"
context.append({'role':'assistant', 'content': "Bienvenido al PizaBot Service, ¿En que puedo ayudarle?"})

while texto_usuario != "salir" or texto_usuario == "":
    texto_usuario = input("CHAT: ")
    if texto_usuario == "salir" or texto_usuario == "":
        continue

    print(texto_usuario)
    context.append(
        {'role':'user', 'content':texto_usuario}
    )

    response = get_completion_from_messages(context, temperature=1)
    print(response)

    context.append(
        {'role':'assistant', 'content': response})

print("FIN DE CHAT")

Qué Pizzas tienen?
 El menú de pizzas incluye las siguientes opciones:
1. Pizza pepperoni - 12.95
2. Pizza de Jamón y Queso - 10.95
3. Pizza con huevo - 11.95
Por favor, indique el número de la pizza que desea para más detalles.

### Response:
 Gracias! Ahora me informas del topping adicional y deseo saber si es pedido al domicilio o para llevar.

Quiero una pizza de Jamón y Queso. ¿Tienen ensaladas?
 Además de las pizzas mencionadas, también tenemos ensaladitas! 
1. Ensalada mixta - 7.25
2. Ensalada Griega - 7.25
Puedes ordenar ambas. ¿Desea algo más?

Si, quiero agregar una Ensalada Griega.
 Ideal! ¡Entonces te estare preparando una Pizza de Jamón y Queso junto con un delicioso ensaladista de Griego. ¿Deseas algo más?

Nada más. Cúanto es el precio final?
 ¡Así! Así es la cuenta: Pizza de Jamón y Queso (10.95) + Ensaladilla Griega (7.25) = $18.20. ¡Vaya a ser un alimento delicioso!


FIN DE CHAT


hola
